In [3]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox

class CiscoConfigApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Cisco Switch Config Generator")
        self.vlans = []
        self.routes = []

        self.create_widgets()

    def create_widgets(self):
        # Hostname Frame
        frame_host = ttk.LabelFrame(self.root, text="Switch Hostname")
        frame_host.pack(fill='x', padx=10, pady=5)
        ttk.Label(frame_host, text="Hostname:").pack(side='left', padx=5)
        self.entry_hostname = ttk.Entry(frame_host)
        self.entry_hostname.pack(side='left', padx=5)

        # Password & Banner Frame
        frame_pw = ttk.LabelFrame(self.root, text="Passwords & Banner (optional)")
        frame_pw.pack(fill='x', padx=10, pady=5)

        self.entries_pw = {}
        labels = [
            ('Enable Secret:', 'enable_secret'),
            ('Console Password:', 'console_password'),
            ('VTY Password:', 'vty_password'),
            ('SSH Username:', 'ssh_username'),
            ('SSH Password:', 'ssh_password'),
            ('Domain Name:', 'domain_name'),
            ('MOTD Banner:', 'motd_banner')
        ]

        for i, (label, key) in enumerate(labels):
            ttk.Label(frame_pw, text=label).grid(row=i, column=0, sticky='w', padx=5, pady=2)
            ent = ttk.Entry(frame_pw, width=40)
            ent.grid(row=i, column=1, sticky='w', padx=5, pady=2)
            self.entries_pw[key] = ent

        # VLANs Frame
        frame_vlan = ttk.LabelFrame(self.root, text="VLANs (ID, Name, IP Address, Subnet Mask)")
        frame_vlan.pack(fill='x', padx=10, pady=5)
        self.frame_vlan_entries = ttk.Frame(frame_vlan)
        self.frame_vlan_entries.pack()

        self.add_vlan_button = ttk.Button(frame_vlan, text="Add VLAN", command=self.add_vlan_row)
        self.add_vlan_button.pack(pady=5)

        # Static Routes Frame
        frame_routes = ttk.LabelFrame(self.root, text="Static Routes (Network, Mask, Next-Hop, Optional AD)")
        frame_routes.pack(fill='x', padx=10, pady=5)
        self.frame_route_entries = ttk.Frame(frame_routes)
        self.frame_route_entries.pack()

        self.add_route_button = ttk.Button(frame_routes, text="Add Route", command=self.add_route_row)
        self.add_route_button.pack(pady=5)

        # Buttons Frame (moved above output box)
        frame_buttons = ttk.Frame(self.root)
        frame_buttons.pack(fill='x', padx=10, pady=5)

        self.btn_generate = ttk.Button(frame_buttons, text="Generate Config", command=self.generate_config)
        self.btn_generate.pack(side='left', padx=5)

        self.btn_clear = ttk.Button(frame_buttons, text="Clear All", command=self.clear_all)
        self.btn_clear.pack(side='left', padx=5)

        # Output Frame
        frame_output = ttk.LabelFrame(self.root, text="Generated Cisco Configuration")
        frame_output.pack(fill='both', expand=True, padx=10, pady=5)

        self.text_output = scrolledtext.ScrolledText(frame_output, height=20)
        self.text_output.pack(fill='both', expand=True)

        # Initialize with 1 VLAN and 1 route
        self.add_vlan_row()
        self.add_route_row()

    def add_vlan_row(self):
        frame = ttk.Frame(self.frame_vlan_entries)
        frame.pack(fill='x', pady=2)

        vlan_id_var = tk.StringVar()
        vlan_name_var = tk.StringVar()
        vlan_ip_var = tk.StringVar()
        subnet_var = tk.StringVar()

        vlan_id_cb = ttk.Combobox(frame, textvariable=vlan_id_var, width=6)
        vlan_id_cb['values'] = [str(i) for i in range(1, 100)]
        vlan_id_cb.pack(side='left', padx=5)

        vlan_name_entry = ttk.Entry(frame, textvariable=vlan_name_var, width=15)
        vlan_name_entry.pack(side='left', padx=5)

        vlan_ip_entry = ttk.Entry(frame, textvariable=vlan_ip_var, width=15)
        vlan_ip_entry.pack(side='left', padx=5)

        subnet_cb = ttk.Combobox(frame, textvariable=subnet_var, width=15)
        subnet_cb['values'] = [
            '255.255.255.0',
            '255.255.255.128',
            '255.255.255.192',
            '255.255.255.224',
            '255.255.255.240',
            '255.255.255.248'
        ]
        subnet_cb.pack(side='left', padx=5)

        btn_remove = ttk.Button(frame, text="Remove", command=lambda: self.remove_vlan_row(frame))
        btn_remove.pack(side='left', padx=5)

        self.vlans.append({
            'frame': frame,
            'vlan_id': vlan_id_var,
            'vlan_name': vlan_name_var,
            'vlan_ip': vlan_ip_var,
            'subnet': subnet_var
        })

    def remove_vlan_row(self, frame):
        for vlan in self.vlans:
            if vlan['frame'] == frame:
                vlan['frame'].destroy()
                self.vlans.remove(vlan)
                break

    def add_route_row(self):
        frame = ttk.Frame(self.frame_route_entries)
        frame.pack(fill='x', pady=2)

        net_var = tk.StringVar()
        mask_var = tk.StringVar()
        nh_var = tk.StringVar()
        ad_var = tk.StringVar()

        net_entry = ttk.Entry(frame, textvariable=net_var, width=15)
        net_entry.pack(side='left', padx=5)

        mask_cb = ttk.Combobox(frame, textvariable=mask_var, width=15)
        mask_cb['values'] = [
            '255.255.255.0',
            '255.255.255.128',
            '255.255.255.192',
            '255.255.255.224',
            '255.255.255.240',
            '255.255.255.248'
        ]
        mask_cb.pack(side='left', padx=5)

        nh_entry = ttk.Entry(frame, textvariable=nh_var, width=15)
        nh_entry.pack(side='left', padx=5)

        ad_entry = ttk.Entry(frame, textvariable=ad_var, width=5)
        ad_entry.pack(side='left', padx=5)

        btn_remove = ttk.Button(frame, text="Remove", command=lambda: self.remove_route_row(frame))
        btn_remove.pack(side='left', padx=5)

        self.routes.append({
            'frame': frame,
            'network': net_var,
            'mask': mask_var,
            'next_hop': nh_var,
            'admin_dist': ad_var
        })

    def remove_route_row(self, frame):
        for route in self.routes:
            if route['frame'] == frame:
                route['frame'].destroy()
                self.routes.remove(route)
                break

    def generate_config(self):
        hostname = self.entry_hostname.get().strip()
        cfg_lines = ["enable", "configure terminal"]
        if hostname:
            cfg_lines.append(f"hostname {hostname}")

        # Optional password & banner settings
        if self.entries_pw['enable_secret'].get().strip():
            cfg_lines.append("service password-encryption")
            cfg_lines.append(f"enable secret {self.entries_pw['enable_secret'].get().strip()}")

        if self.entries_pw['console_password'].get().strip():
            cfg_lines.extend([
                "line con 0",
                f" password {self.entries_pw['console_password'].get().strip()}",
                " login",
                " logging synchronous",
                "exit"
            ])

        if self.entries_pw['vty_password'].get().strip():
            cfg_lines.extend([
                "line vty 0 15",
                f" password {self.entries_pw['vty_password'].get().strip()}",
                " login",
                " logging synchronous",
                "exit"
            ])

        if self.entries_pw['domain_name'].get().strip():
            cfg_lines.append(f"ip domain-name {self.entries_pw['domain_name'].get().strip()}")

        if self.entries_pw['motd_banner'].get().strip():
            cfg_lines.append(f"banner motd #{self.entries_pw['motd_banner'].get().strip()}#")

        if self.entries_pw['ssh_username'].get().strip() and self.entries_pw['ssh_password'].get().strip():
            cfg_lines.append(f"username {self.entries_pw['ssh_username'].get().strip()} privilege 15 secret {self.entries_pw['ssh_password'].get().strip()}")
            cfg_lines.extend([
                "line vty 0 15",
                " transport input ssh",
                " login local",
                "exit",
                "crypto key generate rsa general-keys modulus 1024"
            ])

        cfg_lines.append("ip routing")
        cfg_lines.append("!")

        for vlan in self.vlans:
            vid = vlan['vlan_id'].get().strip()
            name = vlan['vlan_name'].get().strip()
            ip = vlan['vlan_ip'].get().strip()
            subnet = vlan['subnet'].get().strip()
            if vid:
                cfg_lines.append(f"vlan {vid}")
                if name:
                    cfg_lines.append(f" name {name}")
                cfg_lines.append("exit")
                if ip and subnet:
                    cfg_lines.append(f"interface vlan {vid}")
                    cfg_lines.append(f" ip address {ip} {subnet}")
                    cfg_lines.append("exit")

        for route in self.routes:
            net = route['network'].get().strip()
            mask = route['mask'].get().strip()
            nh = route['next_hop'].get().strip()
            ad = route['admin_dist'].get().strip()
            if net and mask and nh:
                line = f"ip route {net} {mask} {nh}"
                if ad:
                    line += f" {ad}"
                cfg_lines.append(line)

        config = "\n".join(cfg_lines)
        self.text_output.delete(1.0, tk.END)
        self.text_output.insert(tk.END, config)

    def clear_all(self):
        self.entry_hostname.delete(0, tk.END)
        for ent in self.entries_pw.values():
            ent.delete(0, tk.END)
        for vlan in self.vlans[:]:
            vlan['frame'].destroy()
            self.vlans.remove(vlan)
        for _ in range(2):
            self.add_vlan_row()
        for route in self.routes[:]:
            route['frame'].destroy()
            self.routes.remove(route)
        self.add_route_row()
        self.text_output.delete(1.0, tk.END)


if __name__ == "__main__":
    root = tk.Tk()
    app = CiscoConfigApp(root)
    root.mainloop()
